# Exercise 2
Today we are going to continue work on pointclouds.
We will work on trying to cluster pointclouds to be able to segment them.
    
If you do not have sklearn installed make sure to **pip install scikit-learn**

In [2]:
import numpy as np
import open3d as o3d
import copy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans, k_means

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def draw_labels_on_model(pcl,labels):
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    print("%s has %d clusters" % (pcl_name, max_label + 1))
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])



## On a cube.
We createa a point cloud using open3d.
Our goal is to segment each side using k means.

In [4]:
pcl_name = 'Cube'
density = 1e4 # density of sample points to create
pcl = o3d.geometry.TriangleMesh.create_box().sample_points_uniformly(int(density))
eps = 0.4
print("%s has %d points" % (pcl_name, np.asarray(pcl.points).shape[0]))
o3d.visualization.draw_geometries([pcl])

Cube has 10000 points
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


If we just use Kmeans out of the box with the pointcloud we will get the following


Note that pressing plus and minus in the viewer will increase/decrease the size of the points in the viewer

In [9]:
km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray
xyz = np.asarray(pcl.points)
labels = km.fit_predict(xyz)
print(labels)
print(xyz.shape)
draw_labels_on_model(pcl, labels)

[1 1 5 ... 5 5 5]
(10000, 3)
Cube has 6 clusters


As we can see we get 6 clusters but they do not span a side.

We try again but this time we instead use the normals of the cube.
The normals for each plane should be parralell with the other normals from said plane.

In [20]:
###
# Code goes here
###



km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=500, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray
pcl.estimate_normals()
print(pcl.normals)
xyz = np.concatenate((np.asarray(pcl.points), np.asarray(pcl.normals)*1.5), axis=1)
labels = km.fit_predict(xyz)
draw_labels_on_model(pcl, labels)

std::vector<Eigen::Vector3d> with 10000 elements.
Use numpy.asarray() to access data.
Cube has 6 clusters


This still does not work, the opposite side will also have normals that point the other way which will parralell.

So to combat this we can attempt to use the xyz coordinates and the normals.

In [1]:
###
# Code goes here
###

d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=1).translate((0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=2).translate((d, -d, 0))
mesh_points = mesh.sample_points_uniformly(int(1e5))

o3d.visualization.draw_geometries([mesh])

km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=500, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray
xyz = np.asarray(mesh_points.points)
labels = km.fit_predict(xyz)
print(labels)
draw_labels_on_model(mesh_points, labels)

NameError: name 'o3d' is not defined

In [42]:
pc = o3d.io.read_point_cloud("TestData/fragment.ply")

km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=500, tol=1e-04, random_state=0)

voxel_size = 0.1
pc_sample = pc.voxel_down_sample(voxel_size)

default_tree = o3d.geometry.KDTreeSearchParamHybrid(radius=10,max_nn=60)

pc.estimate_normals()

fpfh = o3d.pipelines.registration.compute_fpfh_feature(pc_sample, default_tree)
# Get the points from the pointcloud as nparray
xyz = np.concatenate((np.asarray(pc.points), np.asarray(pc.normals)*1.5), axis=1)
labels = km.fit_predict(xyz)
draw_labels_on_model(pc, labels)

Cube has 6 clusters


In [54]:
#eps (float) – Density parameter that is used to find neighbouring points.
eps = 0.1

#min_points (int) – Minimum number of points to form a cluster.
min_points = 2

labels = np.array(pc.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
print(labels)
draw_labels_on_model(pc, labels)

Cube has 1 clusters


## Exercises

### A) K means continued.

Combine the point cloud points (xyz) with the normals and do k-means.

```xyz_n = np.concatenate((xyz, normals), axis=1)```

Do you get better clusters?
Why would adding the normals help?

### B) 
Try weighting either the points or normals by scaling them by some factor, can segment each of the faces of the cube?
### C)
Try to cluster all the different shapes using k means.
```{python}
d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=1).translate(
    (0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=2).translate(
    (d, -d, 0))
mesh.sample_points_uniformly(int(1e5)), 0.5
```

### D)
Now try with the pointcloud in "pointclouds/fragment.ply"
Are you able to cluster the pointcloud?

What features here would it make sense to cluster?
- fpfh features?
- xyz
- normals 
- colors

Are you able to get clusters that make sense? Why?

### E)
Use the built in cluster_dbscan algorithm.
Tweak the parameters and see what you get out.

Attempt on the combined figures and on "fragment.ply"
```{Python}
#eps (float) – Density parameter that is used to find neighbouring points.
eps = 0.02

#min_points (int) – Minimum number of points to form a cluster.
min_points = 10

labels = np.array(pcl.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
```
